In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

# Load the MRI image
image_path = '3_T2_MRI_SWI_BFC_50mm_HM_rsCMB_V1.nii.gz'  # Change this to the path of your MRI image
mri_image = nib.load(image_path)
mri_data = mri_image.get_fdata()

# Iterate through each slice and save it as PNG
for i in range(mri_data.shape[2]):  # Assuming the slice dimension is along the z-axis
    slice_data = mri_data[:, :, i]  # Extract the slice data

    # Plot and save the slice as PNG
    plt.imshow(slice_data, cmap='gray')
    plt.axis('off')  # Remove axis save 
    plt.savefig(f'slice_{i}.png', bbox_inches='tight', pad_inches=0)
    plt.close()


This is the Proccess where i should begin with 

In [1]:
import os
import nibabel as nib
import numpy as np

# Define the path to the data folder
data_folder = 'data'

# Load and preprocess the NIfTI files
preprocessed_data = []

for filename in os.listdir(data_folder):
    if filename.endswith('.nii.gz'):
        filepath = os.path.join(data_folder, filename)
        img = nib.load(filepath)
        # Perform preprocessing steps here (e.g., resampling, normalization)
        # Add the preprocessed image to the list
        preprocessed_data.append(img.get_fdata())

# Convert the preprocessed data to a numpy array
preprocessed_data = np.array(preprocessed_data)

In [2]:
import os
import nibabel as nib
import numpy as np

# Define the path to the data folder
data_folder = 'data'

# Load and preprocess the NIfTI files
preprocessed_data = []

for filename in os.listdir(data_folder):
    if filename.endswith('.nii.gz'):
        filepath = os.path.join(data_folder, filename)
        img = nib.load(filepath)
        
        # Extract the image data
        img_data = img.get_fdata()
        
        # Convert the image to grayscale
        if img_data.shape[-1] == 3:  # Check if the image has 3 channels
            # Convert RGB to grayscale using the luminance method (weighted sum of RGB channels)
            img_data_gray = np.dot(img_data[...,:3], [0.2989, 0.5870, 0.1140])
            
            # Rescale pixel values to the range [0, 1]
            img_data_gray /= img_data_gray.max()
            
            # Append the grayscale image to the preprocessed_data list
            preprocessed_data.append(img_data_gray)
        else:
            # If the image is already grayscale, no conversion is needed
            preprocessed_data.append(img_data)

# Convert the preprocessed data to a numpy array
preprocessed_data = np.array(preprocessed_data)

# Check if the images are grayscale
if preprocessed_data.shape[-1] == 1:
    print("The images are grayscale.")
else:
    print("The images are not grayscale.")


The images are not grayscale.


In [3]:
from sklearn.model_selection import train_test_split

# Split the preprocessed data into training, validation, and testing sets
x_train, x_temp = train_test_split(preprocessed_data, test_size=0.2, random_state=42)
x_val, x_test = train_test_split(x_temp, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", x_train.shape)
print("Validation data shape:", x_val.shape)
print("Testing data shape:", x_test.shape)


Training data shape: (40, 176, 256, 80)
Validation data shape: (5, 176, 256, 80)
Testing data shape: (5, 176, 256, 80)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import MeanSquaredError

# Define input shape
input_shape = (176, 256, 80, 3)  # Assuming RGB images, change the number of channels accordingly
# Define VAE model architecture
latent_dim = 8
# Encoder
encoder_input = layers.Input(shape=input_shape)
x = layers.Conv3D(32, kernel_size=3, activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x)
x = layers.Conv3D(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x)
x = layers.Conv3D(128, kernel_size=3, activation='relu', padding='same')(x)
x = layers.MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x)
x = layers.Flatten()(x)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# Reparameterization trick to sample from the latent space
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.random.normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(11 * 16 * 5 * 128, activation='relu')(decoder_input)
x = layers.Reshape((11, 16, 5, 128))(x)
x = layers.Conv3DTranspose(128, kernel_size=3, activation='relu', padding='same')(x)
x = layers.UpSampling3D((2, 2, 2))(x)
x = layers.Conv3DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)
x = layers.UpSampling3D((2, 2, 2))(x)
x = layers.Conv3DTranspose(32, kernel_size=3, activation='relu', padding='same')(x)
x = layers.UpSampling3D((2, 2, 2))(x)
decoder_output = layers.Conv3DTranspose(3, kernel_size=3, activation='sigmoid', padding='same')(x)  # Adjust output channels

# Define the VAE model
encoder = Model(encoder_input, [z_mean, z_log_var, z], name='encoder')
decoder = Model(decoder_input, decoder_output, name='decoder')
vae_output = decoder(encoder(encoder_input)[2])
vae = Model(encoder_input, vae_output, name='vae')

# Define VAE loss
def vae_loss(x, x_decoded_mean):
    xent_loss = MeanSquaredError()(x, x_decoded_mean)
    kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
    return xent_loss + kl_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=vae_loss)
# Reshape the training, validation, and testing data
# Make sure your data has the correct shape (176, 256, 80, 3)

x_train = np.reshape(x_train, (-1, 176, 256, 80, 3))
x_val = np.reshape(x_val, (-1, 176, 256, 80, 3))
x_test = np.reshape(x_test, (-1, 176, 256, 80, 3))

# Define number of epochs and batch size
epochs = 20
batch_size = 4
# Train the VAE model
history = vae.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, x_val))